In [1]:
from aocd import get_data, submit
from dataclasses import dataclass

In [2]:
example_data = "$ cd /\n$ ls\ndir a\n14848514 b.txt\n8504156 c.dat\ndir d\n$ cd a\n$ ls\ndir e\n29116 f\n2557 g\n62596 h.lst\n$ cd e\n$ ls\n584 i\n$ cd ..\n$ cd ..\n$ cd d\n$ ls\n4060174 j\n8033020 d.log\n5626152 d.ext\n7214296 k"
real_data = get_data(day=7, year=2022)

In [3]:
@dataclass()
class File:
    size: int
    name: str

    def is_file(self):
      return True

@dataclass()
class Directory:
    parent: object
    name: str
    content: dict

    def is_file(self):
        return False

rootDirectory = Directory(None, "/", {})
currentDirectory = rootDirectory

for command_output in real_data.split("$")[1:]:
    command_output_split = command_output.strip().split("\n")
    command = command_output_split[0].strip().split(" ")
    if command[0] == "ls":
      for output in command_output_split[1:]:
        content = output.strip().split(" ")
        if content[0] == "dir":
          currentDirectory.content[content[1]] = Directory(currentDirectory, content[1], {})
        else:
          currentDirectory.content[content[1]] = File(int(content[0]), content[1])
    elif command[0] == "cd":
      if command[1] == "/":
        currentDirectory = rootDirectory
      elif command[1] == "..":
          currentDirectory = currentDirectory.parent
      else:
        currentDirectory = currentDirectory.content[command[1]]
    else:
      raise ValueError("Unknown command")

In [4]:
def getDirectorySize(directory:Directory, sizes: []):
   size = 0
   for item in directory.content.values():
     if item.is_file():
       size = size + item.size
     else:
       size = size + getDirectorySize(item, sizes)
   sizes.append(size)
   return size

In [5]:
sizes = []
size_root = getDirectorySize(rootDirectory, sizes)
print(f"Sum of sizes below 100000: {sum([size for size in sizes if size <=100000])}")

print(f"Size of directory to be deleted: {min([size for size in sizes if size >= 30000000-(70000000-size_root)])}")

Sum of sizes below 100000: 1141028
Size of directory to be deleted: 8278005
